# dict -> pandas.DataFrame 하기

현재 user_user_training_matrix는 dict형식임.
이를 DataFrame으로 바꾸어주어야함.

In [2]:
import pickle
import collections
import numpy as np
import pandas as pd

In [5]:
with open("./user_user_training_matrix_2000.pickle", "rb") as f:
    user_user_training_matrix = pickle.load(f)

In [7]:
print(type(user_user_training_matrix))
print(len(user_user_training_matrix))

<class 'dict'>
4000000


In [9]:
columns = ('education', 'religion', 'bodyType', 'diet', 'age', 'smokes', 'drinks', 'pets')
index = pd.MultiIndex.from_tuples(user_user_training_matrix.keys())
df = pd.DataFrame.from_dict(user_user_training_matrix, orient='index', columns=columns)
df.head(5)

,education,religion,bodyType,diet,age,smokes,drinks,pets
"(8195, 8195)",1.0,1.0,0.869592,1.0,1.00,1.0,1.0,2
"(8195, 8202)",0.5,0.5,0.934448,1.0,0.00,0.0,0.0,2
"(8195, 13)",0.5,0.5,0.787254,1.0,0.25,1.0,1.0,0
"(8195, 8207)",1.0,0.5,0.869592,1.0,0.25,1.0,1.0,0
"(8195, 17)",1.0,0.5,0.869592,1.0,0.50,1.0,1.0,2


In [17]:
user_index = []
for i in df.index:
    user_index.append(i[0])

In [19]:
user_unique = set(user_index)
print(len(user_unique))

2000


In [31]:
user_unique_str = []
for user in user_unique:
    user_unique_str.append(str(user))

In [11]:
df.to_numpy().shape

(4000000, 8)

In [12]:
with open('../pickle_files/essay_data.pickle', "rb") as f:
    essay_data = pickle.load(f)

In [13]:
essay_data.shape

(15001, 10)

In [28]:
essay_data.index

Index(['-1', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '14990', '14991', '14992', '14993', '14994', '14995', '14996', '14997',
       '14998', '14999'],
      dtype='object', length=15001)

In [35]:
essay_train_label = essay_data.loc[user_unique_str]

In [36]:
essay_train_label

,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
8226,,"currently, i'm living in the south bay working...",i have always been a bit of a math/science gee...,"long, flowing golden locks waving against glis...",i love fantasy books. just recently read jim b...,in no particular order: 1) joss whedon 2) sta...,,typical friday?! i guess i try not to have a t...,"i play roleplaying games. there, i said it! i ...","...you are intelligent and easygoing. really, ..."
14441,i recently moved to the bay area from the midw...,apparently starting over in the strange new wo...,"ballroom and swing dancing, wine, getting lost...",,"i used to love tv, but i've certainly narrowed...",,- i think a lot about how i ended up all the w...,"currently sitting home, relaxing with a glass ...",i'm terribly lonely in this town. i find the b...,you are also just looking for someone to hang ...
10005,i was born and raised in taiwan and came to th...,"working at a pretty awesome startup, enjoying ...",,"good question, i've never asked","good will hunting, pulp fiction, social networ...","dreams, iphone, sunshine, friends, traveling, ...","who's gonna win the nba championship, why the ...",,sometimes i cry watching sports and movies,you want to
9442,i'm new to this so it will take time 2 do my p...,living it to the fullest,everything i put my mind to,my smile,"fav movies are 300,gladiator and coming to ame...",,,,nothing at all unless u ask i won't tell,you think i'm cute
2915,i am currently an online student at arizona st...,i am currently majoring in psych at asu. i hop...,i have played softball basically all my life a...,either my natural blonde hair or blue eyes,i like twilight books and movies i also love c...,"my cell phone, my family, my friends, my lapto...",what i really want to do with my life,hopefully hanging out with some friends but la...,i have some regrets but i plan on making the m...,you are no older than 26 and are looking to ha...
...,...,...,...,...,...,...,...,...,...,...
13969,ok! anyone down for a beer and maybe a movie o...,overall goal right now is to play outside and ...,talking to overseas helpdesk technicians...,,,rock climbing water..... sushi laptop nerds ro...,,,,you want to grab a beer
7966,three months ago i moved to san francisco from...,"for work, i'm in the it department of an insur...",,the lack of effort i put into this ok cupid pr...,"movies: recently, i thought cabin in the woods...",friends/family internet cell phone microwave n...,where i'm going to eat for lunch.,alive.,"i have a really embarrassing ""everyday"" playlist.",you would like to know which country hosts the...
11211,rock solid abs. rock. solid. rock freaking sol...,probably could have played pro hockey and pro ...,"living, loving, lounging.","biceps, followed by my triceps.",the ones i make.,"the six wheels on my wide body, extended cab p...",the security of my offshore bank accounts.,keeping it real. real real.,i secretly love my life!!!!,you appreciate sarcasm
10159,i grew up in the bay area and i'm half chinese...,graduate student!,paying attention to detail. ...,,"books: harry potter! the secret, how to win fr...",,,,,


In [107]:
essay_train_label.iloc[2][0]

"i was born and raised in taiwan and came to the states in 2006. i spent 4 years in boston and got a master's degree in marketing communication but ended up becoming a software engineer. in 2010, i drove across the country to sf to start a new job.  the last five years have been an amazing experience and most of time i enjoy being a nomad. i wanna keep working hard and see where the journey may take me.  i consider myself introspective, observant and curious about all things new. i like to stay active, i play basketball and ping pong, swim, snowboard, and want to play more tennis.  i like people who are funny, genuine and open-minded and hope that's how people feel about me."

In [97]:
essay_train_label.shape

(2000, 10)

In [38]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

model_name = 'bert-base-nli-mean-tokens'
model = SentenceTransformer(model_name)

In [44]:
sentence_vecs = [0]*10
for i in range(10):
    sentence_vecs[i] = model.encode(essay_train_label['essay' + str(i)])

In [48]:
cos_result = [0]*10
for i in range(10):
    cos_result[i] = cosine_similarity(sentence_vecs[i], sentence_vecs[i])

In [51]:
cos_result[0].shape

(2000, 2000)

In [52]:
cos_result_sum = sum(cos_result)

In [53]:
cos_result_sum.shape

(2000, 2000)

In [54]:
cos_result_sum

array([[10.000001 ,  3.2580059,  2.1726851, ...,  2.7191434,  2.0059648,
         4.570388 ],
       [ 3.2580059, 10.       ,  3.5819936, ...,  2.1302924,  3.9325664,
         3.3235745],
       [ 2.1726851,  3.5819936, 10.000001 , ...,  2.8142097,  4.580935 ,
         3.8609786],
       ...,
       [ 2.7191434,  2.1302924,  2.8142097, ..., 10.       ,  2.9124048,
         2.752478 ],
       [ 2.0059648,  3.9325664,  4.580935 , ...,  2.9124048, 10.000001 ,
         3.6945105],
       [ 4.570388 ,  3.3235745,  3.8609786, ...,  2.752478 ,  3.6945105,
        10.       ]], dtype=float32)

In [93]:
np.where(cos_result_sum[7] > 6)

(array([   7,   37,   58,   73,   82,   86,   97,   99,  101,  105,  120,
         122,  128,  139,  150,  151,  154,  175,  176,  177,  178,  179,
         191,  197,  206,  207,  212,  219,  233,  234,  246,  248,  253,
         264,  286,  291,  296,  303,  320,  328,  349,  355,  365,  369,
         383,  389,  396,  411,  424,  432,  433,  438,  439,  465,  469,
         491,  510,  512,  516,  523,  529,  533,  539,  547,  556,  558,
         570,  573,  587,  593,  597,  599,  603,  624,  633,  634,  638,
         644,  652,  675,  681,  683,  684,  690,  692,  702,  704,  712,
         735,  741,  755,  757,  764,  777,  779,  785,  787,  793,  802,
         810,  812,  833,  851,  852,  860,  876,  887,  902,  910,  918,
         920,  925,  931,  933,  935,  946,  961,  966,  970,  978,  983,
         990, 1006, 1007, 1023, 1027, 1057, 1062, 1067, 1076, 1082, 1084,
        1088, 1118, 1122, 1140, 1141, 1146, 1156, 1165, 1176, 1185, 1204,
        1219, 1220, 1224, 1240, 1248, 

In [94]:
cos_result_sum.shape

(2000, 2000)

In [95]:
print(type(cos_result_sum))

<class 'numpy.ndarray'>


### bert로 계산한 유사도 파일 저장

In [108]:
with open("./cos_result_sum_2000.pickle", "wb") as f:
    pickle.dump(cos_result_sum, f)